# PTST with non-zero thickness ligament 

In [1]:
import numpy as np

To combine axi-symmetric stress state with the fatigue degradation ligament
it is necessary to address the issue of correct representation of the uniform elastic
state.

## Option #1 - application of slide for reduced strain state

To correctly reproduce the axisymmetric strain and stress state, consider the strain tensor
projections
$$
  e_a = n_a \varepsilon_{ab}
$$
Assuming that the axis of symmetry is aligned with $x$ axis,
the normal to the interface is given as $n_a = [0, 1, 0]$. 
Thus, the normal and tangential strain components can be 
detached from the strain tensor by setting
$$
  e_N = \delta_{Na} e_a, e_T = \delta_{Ta} e_a
$$
where
$$
\delta_{Na} = [0, 1, 0], \;
\delta_{Ta} = 
\left[
\begin{array}{lll}
 0, & 1, & 0 \\
 0, & 0, & 1
\end{array}
\right].
$$

$$
  e_N = \varepsilon_{11}, e_T = \varepsilon_{10}.
$$
Note that the strain component $\varepsilon_{00}$ is disregarded. Is it an acceptable assumption? The stress along the ligament is implicitly assumed zero.

$$
\begin{array}{ccc}
 \varepsilon_{00} &  \varepsilon_{T} & 0 \\
 \varepsilon_{T} &  \varepsilon_{N} & 0 \\
 0 &  0 & \varepsilon_{\theta}
\end{array}
$$

$$
\begin{array}{ccc}
 \sigma_{00} &  \sigma_{T} & 0 \\
 \sigma_{T} &  \sigma_{N} & 0 \\
 0 &  0 & \sigma_{\theta}
\end{array}
$$

# Damage plasticity for decohesion/compression along dissipative plane

## Normal strain component

### Kinematic constraint

The normal strain vector acting in the direction $n_a$ is given as

$$
  \varepsilon_N = n_a n_b \varepsilon_{ab}
$$

The strain tensor $\varepsilon_{N_{ab}}$ equivalent to the normal strain $\varepsilon_N$ acting in the direction $n_a$ is obtained using the principle of virtual work. Considering a virtual stress tensor $\delta \sigma_{ab}$, the corresponding virtual stress $\sigma_N$ acting in direction $n_a$ is obtained as the projection

$$
  \delta \sigma_N = n_a n_b \delta \sigma_{ab}
$$

The representation of the normal strain $\varepsilon_N$ within a tensor can be established by requiring

$$
  \varepsilon_{N_{ab}} \delta \sigma_{ab} = \varepsilon_N \delta \sigma_N
$$

By substituting for $\delta \sigma_{N}$ we obtain

$$
  \varepsilon_{N_{ab}} = n_a n_b \varepsilon_N.
$$

The direct relation between strain tensor $\varepsilon_N$ and the fraction associated with the direction $n_a$ is obtained as

$$
  \varepsilon_{N_{ef}} = n_e n_f \varepsilon_N = n_e n_f n_a n_b \varepsilon_{ab}.
$$

**Remark:** The complementary part of the strain tensor can be extracted as

$$
  \varepsilon_{\setminus N_{ab}} = \varepsilon_{ab} - \varepsilon_{N_{ab}}
$$

By substituting for $\varepsilon_N$ and applying an identity rank four tensor, this can be rewritten as

$$
  \varepsilon_{\setminus N_{ef}} = (I_{efab} - n_e n_f n_a n_b) \varepsilon_{ab}
$$

**Check - tensor properties:**
Are the invariants of $\varepsilon_{ab}$ equivalent to the sum of invariants obtained separately for $\varepsilon_{\setminus N_{ab}}$ and $\varepsilon_{N_{ab}}$?

**Check - equivalence of free energy:** Is the non-dissipative part of Helmholtz free energy defined on $\varepsilon_{\setminus N}$ equivalent to the difference between 
the free energy associated with the full strain tensor $\varepsilon_{ab}$ and the normal strain $\varepsilon_N$?

$$
 \rho \psi_{\varepsilon}^\mathrm{e} - \rho \psi_{\varepsilon_N}^\mathrm{e} 
 =
 \rho \psi_{\varepsilon_{\setminus N}}^\mathrm{e}
$$

The left hand side is expressed as

\begin{align}
 \rho \psi_{\varepsilon}^\mathrm{e} - \rho \psi_{\varepsilon_N}^\mathrm{e} 
 &=
 \frac{1}{2} D_{abcd} \varepsilon_{ab} \varepsilon_{cd} - \frac{1}{2} D_{abcd} \varepsilon_{N_{ab}} \varepsilon_{N_{cd}} \\
 &=
 \frac{1}{2} D_{abcd} \varepsilon_{ab} \varepsilon_{cd} - \frac{1}{2} D_{abcd} \, n_a n_b n_e n_f \varepsilon_{ef} \, n_c n_d n_g n_h  \varepsilon_{gh} \\
 &=
 \frac{1}{2} D_{abcd} (I_{abef}  I_{cdgh} - n_a n_b n_e n_f n_c n_d n_g n_h ) \varepsilon_{ef} \varepsilon_{gh} 
\end{align}

while for the right hand side the expression is obtained

\begin{align}
\rho \psi_{\setminus N}^\mathrm{e} &= \frac{1}{2} D_{abcd} \varepsilon_{\setminus N_{ab}} \varepsilon_{\setminus N_{cd}} \\
 &= 
 \frac{1}{2} D_{abcd} \, (I_{abef} - n_a n_b n_e n_f) \varepsilon_{ef} \, (I_{cdgh} - n_c n_d n_g n_h) \varepsilon_{gh} \\ 
 &=
 \frac{1}{2} D_{abcd} \, (I_{abef} - n_a n_b n_e n_f) (I_{cdgh} - n_c n_d n_g n_h) \, \varepsilon_{ef} \varepsilon_{gh} \\
 &= 
 \frac{1}{2} D_{abcd} \, (I_{abef} I_{cdgh} - I_{cdgh} n_a n_b n_e n_f - I_{abef} n_c n_d n_g n_h + n_a n_b n_e n_f n_c n_d n_g n_h) \, \varepsilon_{ef} \varepsilon_{gh} \\
\end{align}

The free energy of the complementary strain is not the same as the difference between 
the free energy associated with the original strain reduced by the free energy of the normal strain, i.e. 

$$
 \rho \psi_{\varepsilon}^\mathrm{e} - \rho \psi_{\varepsilon_N}^\mathrm{e} 
 \neq 
 \rho \psi_{\varepsilon_{\setminus N}}^\mathrm{e}\\
 \frac{1}{2} D_{abcd} \varepsilon_{ab} \varepsilon_{cd} - \frac{1}{2} D_{abcd} \varepsilon_{N_{ab}} \varepsilon_{N_{cd}} \neq
 \frac{1}{2} D_{abcd} \varepsilon_{\setminus N_{ab}} \varepsilon_{\setminus N{cd}}
$$

### Helmholtz free energy

Given a strain tensor $\varepsilon_{ab}$ and a vector $n_a$ defining the orientation of the dissipative plane, the free energy including elastic bulk material and dissipative plane with damage $\omega_{N}$ and plastic strain $\varepsilon_N^p$ can be written as

$$
\newcommand{\sd}[3]{   {#1}_{#2}^{\mathrm{#3}}
}
\rho \sd{\psi}{N}{}
= \frac{1}{2} D_{abcd} \varepsilon_{ab} \varepsilon_{cd}
-
 \frac{1}{2} D_{abcd} \varepsilon_{N_{ab}} \varepsilon_{N_{cd}}
+
 \frac{1}{2} (1- \omega_N) D_{abcd} \, n_a n_b (n_e n_f \varepsilon_{ef} - \varepsilon^\mathrm{p}_N) \, n_c n_d ( n_g n_h \varepsilon_{gh} - \varepsilon^\mathrm{p}_N)
$$

The macroscopic strain is then obtained as

\begin{align}
 \sigma_{ab} &= \frac{\partial \rho \psi_N}{\partial \varepsilon_{ab}} \\
  &=
  D_{abcd} \varepsilon_{cd} - D_{abcd} n_a n_b n_e n_f n_c n_d n_g n_h \varepsilon_{gh} 
  +
  (1-\omega_N) D_{abcd} n_a n_b n_e n_f n_c n_d (n_g n_h \varepsilon_{gh} - \varepsilon^\mathrm{p}_N)
\end{align}

**Remark - elastic case:** By setting $\omega_N = 0$ and $\varepsilon^\mathrm{p}_N = 0$, the elastic matrix $D_{abcd}$ is recovered.

**Remark - damage in N direction:** By setting $\varepsilon^\mathrm{p}_N = 0$, the damage variable is introduced in the direction $N$ normal to the plane $n_a$. The damage tensor $\beta_{abef}$ can be derived by setting
\begin{align}
 \beta_{abef} = I_{abef} - \phi n_a n_b n_e n_f
\end{align}
where $\phi = \sqrt{1 - \omega}$. Then, the effective damage stiffness can be expressed as 
$$
  D^{\omega}_{abcd} = \beta_{abef} D_{efgh} \beta_{cdgh}
$$

**Remark - thermodynamic forces:** The corresponding thermodynamic forces $\sigma^\mathrm{p}_{cd}$ and energy release rate $Y_N$

$$
  \sigma^\mathrm{p}_{N} = \frac{\partial \rho \psi_N}{\partial \varepsilon_N^\mathrm{p}} =  (1 - \omega_N) D_{abcd} n_a n_b n_c n_d \varepsilon^\mathrm{p}_N \\
  Y_N = -\frac{\partial \rho \psi_N}{\partial \omega} = \frac{1}{2} D_{abcd} n_a n_b (n_e n_f \varepsilon_{ef} - \varepsilon_N^\mathrm{p})
  n_c n_d ( n_g n_h \varepsilon_{gh} - \varepsilon^\mathrm{p}_N)
$$

**Questions:** 
 - How does the above derivation relate to spectral decomposition?
 - Is there any relation to orientation distribution functions (ODF)?
 - Is spurious energy dissipation excluded?
 - What are the properties of the secant material tensor? Which symmetries are lost? Is this admissible? 
 - It is usual to ascribe dissipative effects to tensor invariant, e.g. J2 plasticity. In analogy to the above
   one could define reversible part of free energy as the difference between the full strain tensor $\varepsilon_{ab}$
   and subtracting the free energy associated with the deviatoric part of strain tensor. Then, the dissipative term would be
   added again acting upon the deviatoric part of the strain tensor.
 - How to inject several dissipative planes/mechanisms? Do they have to be orthogonal? Is the assumption  
   for such a procedure spectral decomposition / irreducible stiffness representation (ODF)?

## Tangential strain and stress 

\begin{align}
  \varepsilon_{T_a} &= \varepsilon_{ab} n_b 
  - n_a \varepsilon_N = \frac{1}{2} 
  (\varepsilon_{ab} n_b + \varepsilon_{ba} n_b)
  - n_a n_b n_c \varepsilon_{bc}\\
  &= \frac{1}{2}\left[
     n_b \delta_{ca} + n_c \delta_{ba}
     - 2 n_b n_c n_a
  \right] \varepsilon_{bc}
\end{align}

$$
\sigma_{T_{ab}} \delta \varepsilon_{ab}
=
\sigma_{T_a} \delta \varepsilon_{T_a}
$$

$$
  \sigma_{T_{ab}} = \sigma_{T_a} \frac{1}{2} \left[
    n_b \delta_{ca} + n_c \delta_{ba}
  \right]
$$

Analogical steps to the above derivation are possible.

In [2]:
eps_ab = np.array([[0.1, 1, 1],
                   [1, 0.8, 1],
                   [1, 1, 0.3]
                   ])
v_a = np.array([1, .1, 0], np.float_)
n_a = v_a / (np.sqrt(np.einsum('a,a', v_a, v_a)))
n_a

array([0.99503719, 0.09950372, 0.        ])

In [3]:
eps_N = np.einsum('a,b,ab', n_a, n_a, eps_ab)
eps_N

0.30495049504950505

In [4]:
eps_N_ab = np.einsum('a,b->ab', n_a, n_a) * eps_N
eps_N_ab

array([[0.30193118, 0.03019312, 0.        ],
       [0.03019312, 0.00301931, 0.        ],
       [0.        , 0.        , 0.        ]])

In [5]:
eps_N2 = np.einsum('a,b,ab', n_a, n_a, eps_N_ab)
eps_N2

0.3049504950495053

In [6]:
eps_ab - eps_N_ab

array([[-0.20193118,  0.96980688,  1.        ],
       [ 0.96980688,  0.79698069,  1.        ],
       [ 1.        ,  1.        ,  0.3       ]])

In [7]:
delta_ab = np.identity(3)
T_abc = 0.5 * (
        np.einsum('a,bc -> abc', n_a, delta_ab) +
        np.einsum('b,ac -> bac', n_a, delta_ab) - 2 *
        np.einsum('a,b,c -> abc', n_a, n_a, n_a)
)

In [8]:
np.einsum('abc,ab->c', T_abc, eps_ab)

array([-0.10442965,  1.04429646,  1.09454091])

$$
 \sigma_{ab} = \frac{\partial \rho \psi_T}{ \partial \varepsilon_{cd} }
$$

$$
\sigma_{ab} = 
 \beta_{abcd}
 D_{cdef}
 \beta_{efgh}
 (\varepsilon_{gh} - \varepsilon_{gh}^p)
$$

$$
 \rho \psi_{n} = \frac{1}{2} n_c n_d \sigma_{cd}  n_a n_b \varepsilon_{ab}
 = \frac{1}{2} n_c n_d D_{cdef} \varepsilon_{ef}  n_a n_b \varepsilon_{ab}
$$